In [4]:
import numpy as np
import random
import torch

a = torch.tensor([1,0,1,0,0,0,1, 0,0,1])
b = [0,2,6,9]
# get the sparse index of a
indices = torch.nonzero(a).squeeze()

print(indices)

tensor([0, 2, 6, 9])
